In [1]:
import torch
from pathlib import Path
from tqdm import tqdm
from PIL import Image
from transformers import BlipForQuestionAnswering, BlipProcessor
from torch.utils.data import DataLoader, Dataset

d:\project\student_resource 3\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
MAX_LENGTH=20
BATCH_SIZE=8
NUM_EPOCHS=20
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
BASE_DIR=Path('D:\\project\\student_resource 3')
%cd {BASE_DIR}

cuda
D:\project\student_resource 3


d:\project\student_resource 3\env\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
model_name = "Salesforce/blip-vqa-base"
model = BlipForQuestionAnswering.from_pretrained(model_name)
processor = BlipProcessor.from_pretrained(model_name)

In [5]:
class VQATestDataset(Dataset):
    def __init__(self, dataframe, processor, max_length=MAX_LENGTH):
        self.dataframe = dataframe
        self.processor = processor
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image = Image.open(row["image_path"]).convert("RGB")
        question = row["question"]
        answer = row["answer"]  # Ground-truth answer for evaluation

        inputs = self.processor(
            text=question,
            images=image,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )

        for key in inputs:
            inputs[key] = inputs[key].squeeze(0)

        return {
            "pixel_values": inputs["pixel_values"],
            "input_ids": inputs["input_ids"],
            "attention_mask": inputs["attention_mask"],
            "question": question,
            "answer": answer,  # For evaluation purposes
        }

In [6]:
import pandas as pd

test_df = pd.read_csv(f"{BASE_DIR}/data/test_vqa.csv")

In [7]:
test_dataset = VQATestDataset(test_df, processor)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
for i in test_dataloader:
    print(i["pixel_values"].shape)
    break

torch.Size([8, 3, 384, 384])


In [8]:
from evaluate import load

metric = load("sacrebleu")

In [10]:
model = model.to(device)
model.eval()

predictions = []
references = []

with torch.no_grad():
    for batch in tqdm(test_dataloader):
        pixel_values = batch["pixel_values"].to(device)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        # Generate predictions
        outputs = model.generate(
            input_ids=input_ids,
            pixel_values=pixel_values,
            attention_mask=attention_mask,
            max_length=MAX_LENGTH,  # Ensure this matches your model configuration
            num_beams=5,  # Beam search for better quality answers
        )

        # Decode predictions and ground-truth answers
        predicted_answer = processor.tokenizer.batch_decode(
            outputs, skip_special_tokens=True
        )
        ground_truth_answer = batch["answer"]

        predictions.append(predicted_answer)
        references.append(ground_truth_answer)  # List of lists for BLEU metric

        metric.add_batch(predictions=predicted_answer, references=ground_truth_answer)

        # print(f"Q: {batch['question'][0]}")
        # print(f"Predicted: {predicted_answer}")
        # print(f"Ground Truth: {ground_truth_answer}\n")

# Evaluate the BLEU score
results = metric.compute()
print(f"BLEU Score: {results['score']}")

100%|██████████| 500/500 [06:55<00:00,  1.20it/s]


BLEU Score: 0.0822000619081989


In [11]:
import numpy as np
from itertools import chain

In [12]:
results_df = pd.DataFrame()
results_df["labels"] = list(chain.from_iterable(references))
results_df["predictions"] = list(chain.from_iterable(predictions))
results_df.head()

,labels,predictions
0,6.5 kilogram,10 lbs
1,96.0 watt,0. 5
2,230.0 millimetre,10 inches
3,10.0 centimetre,6 inches
4,60.0 watt,120


In [13]:
results_df.to_csv(f"{BASE_DIR}/results/raw_blip.csv", index=False)